In [1087]:
# Import dependencies
import pandas as pd
import os
from dotenv import load_dotenv    # from Karen's or Khaled's code
from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import select      # Not used
import psycopg2
import csv   # Not used
import numpy as np
from pprint import pprint
import json
import logging
import boto3        # AWS SDK for Python
from botocore.exceptions import ClientError


# import gzip    # Not used

In [1088]:

url_metar_gz="https://aviationweather.gov/data/cache/metars.cache.csv.gz"
url_TAF_gz="https://aviationweather.gov/data/cache/tafs.cache.csv.gz"
url_airsigmets_gz="https://aviationweather.gov/data/cache/airsigmets.cache.csv.gz"

__To retrieve METAR data__

In [1089]:
# Fetch, load, and decompress the data relative to metar
metar_data_df = pd.read_csv(url_metar_gz, header=5, compression='gzip')
metar_data_df.head()

,raw_text,station_id,observation_time,latitude,longitude,temp_c,dewpoint_c,wind_dir_degrees,wind_speed_kt,wind_gust_kt,...,maxT24hr_c,minT24hr_c,precip_in,pcp3hr_in,pcp6hr_in,pcp24hr_in,snow_in,vert_vis_ft,metar_type,elevation_m
0,CYKL 020732Z AUTO 24016G22KT 6SM -SN FEW016 M1...,CYKL,2024-01-02T07:32:00Z,54.8030,-66.8040,-12.0,-14.0,240,16.0,22.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,507.0
1,KRHP 020731Z AUTO 00000KT 10SM CLR M03/M06 A30...,KRHP,2024-01-02T07:31:00Z,35.1939,-83.8618,-3.5,-6.4,0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,METAR,515.0
2,KJKJ 020731Z AUTO 28009KT 10SM CLR M03/M06 A29...,KJKJ,2024-01-02T07:31:00Z,46.8404,-96.6545,-3.0,-6.0,280,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,METAR,280.0
3,CYYN 020731Z AUTO 22009KT 3/8SM SN FEW001 M08/...,CYYN,2024-01-02T07:31:00Z,50.2920,-107.6910,-8.0,-10.0,220,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,814.0
4,CYXS 020731Z AUTO 01005KT 1/2SM FZFG SCT001 M0...,CYXS,2024-01-02T07:31:00Z,53.8840,-122.6770,-6.0,-7.0,10,5.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SPECI,685.0


In [1090]:
# Convert the dataframe to a JSON format
# metar_json = json.loads(json.dumps(list(metar_data_df.T.to_dict().values())))   # https://stackoverflow.com/questions/39257147/convert-pandas-dataframe-to-json-format answer #10 Amir.S
# pprint(metar_json, sort_dicts=False)  # the pprint process is slow (10.5sec)

In [1091]:
# pprint(metar_json, sort_dicts=False)

In [1092]:
# Convert some columns to INT
# cols=['wind_dir_degrees','wind_speed_kt','wind_gust_kt','cloud_base_ft_agl','cloud_base_ft_agl.2',\
#     'cloud_base_ft_agl.3','vert_vis_ft','elevation_m']
# metar_data_df[cols]=metar_data_df[cols].apply(pd.to_numeric, errors='coerce', downcast='integer', axis=1) # Does not appear to convert to INT, but to FLOAT64
# metar_data_df[cols]=metar_data_df[cols].astype(int)   # Cannot convert NaN

In [1093]:
# metar_data_df.columns

In [1094]:
# metar_data_df.info()

In [1095]:
# convert the dataframe to a dictionary then to a JSON string
metar_string=json.dumps(list(metar_data_df.T.to_dict().values()))

# open the file in write mode
# output_path = os.path.join("Resources", "metar_data.json")  # To be removed if logic.js cannot read from Resources
output_path = os.path.join("static", "metar_data.json")
with open(output_path, "w") as file:
    # write the JSON string to the file
    file.write(metar_string.replace("NaN","null"))

# file is automatically closed after the with block

In [1096]:
# To save the dataframe as a csv file for future import to database (not needed anymore)
# output_path2 = os.path.join("", "metar_data.csv")
# metar_data_df.to_csv(output_path2, index=False)

# file is automatically closed after the with block

__To refresh the metar table in the Render database__

In [1097]:
# Empty the metar table

load_dotenv()
db_url = os.environ.get("link_render")
connection = psycopg2.connect(db_url)
cursor = connection.cursor()
cursor.execute("DELETE FROM metar;")    # Deletes all the rows but keep the table
connection.commit()
cursor.close()
connection.close()

In [1098]:
# Repopulate the empty table from a json file
load_dotenv()
db_url = os.environ.get("link_render")
connection = psycopg2.connect(db_url)
cursor = connection.cursor()

cursor.execute("set search_path to public") # https://dba.stackexchange.com/questions/268365/using-python-to-insert-json-into-postgresql

with open(output_path) as file:
    data = file.read()

query_sql = """
INSERT INTO metar SELECT * FROM
json_populate_recordset(NULL::metar, %s);
"""

cursor.execute(query_sql, (data,))
connection.commit()

__To download a new json file made of the joining of airport data and weather data__ (not used anymore: To be deleted)

In [1099]:
# # download the output of a query across multiple tables with all the active public airport with or without weather information.
# # Will be used to populate the makers that do not have METAR information.
# # There is only one row per airport so only one runway is listed even if the airport as more.
# # More parameters can be added to complete the info on the popup window.

# load_dotenv()
# db_url = os.environ.get("link_render")

# # query="""
# #     SELECT DISTINCT ON (arpt_id)
# # 	arpt_id, icao_id, metar.station_id, arpt_name, apt_rwy.rwy_id, lat_decimal, long_decimal, metar.observation_time, metar.wind_speed_kt, metar.flight_category, metar.raw_text
# #     FROM apt_rwy
# #     JOIN apt_base ON apt_base.site_no = apt_rwy.site_no
# #     FULL JOIN metar ON metar.station_id = apt_base.icao_id
# #     WHERE facility_use_code='PU' AND site_type_code='A' AND arpt_status='O';
# #     """

# query="""
# SELECT DISTINCT ON (arpt_id)
#     arpt_id, icao_id, metar.station_id, arpt_name, apt_rwy.rwy_id, lat_decimal, metar.latitude, long_decimal, metar.longitude, metar.observation_time, metar.wind_speed_kt, metar.flight_category, metar.raw_text, elev, metar.visibility_statute_mi, metar.cloud_base_ft_agl
# FROM apt_rwy
# JOIN apt_base ON apt_base.site_no = apt_rwy.site_no
# FULL JOIN metar ON (RIGHT(metar.station_id, LENGTH(metar.station_id) - 1)) = apt_base.arpt_id
# WHERE facility_use_code='PU' AND site_type_code='A' AND arpt_status='O' AND
# CASE
# 	WHEN metar.station_id IS NOT NULL
# 	THEN (@(lat_decimal - metar.latitude) <1) AND (@(long_decimal - metar.longitude) <1) AND site_type_code='A'
	
# 	WHEN metar.station_id IS NULL
# 	THEN site_type_code='A'
# END
# """

# engine=create_engine(db_url)
# with engine.begin() as conn:
#     results=conn.execute(
#         text(query)
#     )

# arpt_weather_query_df = pd.DataFrame(results)

# # Save the df as a CSV file. Might not be needed if we only use JSON    TO BE REMOVED?
# # arpt_weather_query_df.to_csv (r'airport_weather_data.csv', index = False) # place 'r' before the path name


# # convert the dataframe to a dictionary then to a JSON string
# arpt_weather_string=json.dumps(list(arpt_weather_query_df.T.to_dict().values()))

# # open the file in write mode
# output_path = os.path.join("", "airport_weather_data.json")
# with open(output_path, "w") as file:
#     # write the JSON string to the file
#     file.write(arpt_weather_string.replace("NaN","null"))

# # file is automatically closed after the with block



In [1100]:
# arpt_weather_query_df.head()

In [1101]:
# pprint(arpt_weather_string)

__To retrieve TAF data__

In [1102]:
# Fetch, load, and decompress the data relative to TAF  (working but not used for now)
# taf_data_df = pd.read_csv(url_TAF_gz, header=5,index_col=False, compression='gzip',dtype='str',low_memory=False)
# taf_data_df.head()

In [1103]:
# Convert the dataframe to a JSON format  (working but not used for now)
# taf_json = json.loads(json.dumps(list(taf_data_df.T.to_dict().values())))   # https://stackoverflow.com/questions/39257147/convert-pandas-dataframe-to-json-format answer #10 Amir.S
# pprint(taf_json, sort_dicts=False)  # the pprint process is slow (10.5sec)

In [1104]:
# # convert the dataframe to a dictionary then to a JSON string  (working but not used for now)
# taf_string=json.dumps(list(taf_data_df.T.to_dict().values()))

# # open the file in write mode
# output_path = os.path.join("Resources", "taf_data.json")
# with open(output_path, "w") as file:
#     # write the JSON string to the file
#     file.write(taf_string.replace("NaN","null"))

# # file is automatically closed after the with block

__To retrieve AIRMET and SIGMET polygons data__

In [1105]:
# Fetch, load, and decompress the data relative to Airmet and Sigmet
airsigmet_data_df = pd.read_csv(url_airsigmets_gz, header=5, compression='gzip', encoding='utf-8')
airsigmet_data_df.head()

,raw_text,valid_time_from,valid_time_to,lon:lat points,min_ft_msl,max_ft_msl,movement_dir_degrees,movement_speed_kt,hazard,severity,airsigmet_type
0,WSUS32 KKCI 020655 SIGC CONVECTIVE SIGMET.....,2024-01-02T06:55:00Z,2024-01-02T08:55:00Z,-107:49;-107:31.79;-106.52:31.78;-106.48:31.75...,NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
1,WSUS31 KKCI 020655 SIGE CONVECTIVE SIGMET 6...,2024-01-02T06:55:00Z,2024-01-02T08:55:00Z,-72.4317:37.2917;-73.7076:36.2246;-74.3245:36....,21000.0,NaN,NaN,CONVECTIVE,LT-MOD,SIGMET,NaN
2,WSUS33 KKCI 020655 SIGW CONVECTIVE SIGMET 2...,2024-01-02T06:55:00Z,2024-01-02T08:55:00Z,-109.9843:32.0135;-108.1224:31.2;-110.6217:31....,28000.0,NaN,NaN,CONVECTIVE,LT-MOD,SIGMET,NaN
3,WAUS45 KKCI 020245 SLCZ WA 020245 AIRMET ZUL...,2024-01-02T02:45:00Z,2024-01-02T09:00:00Z,-106.2056:35.0385;-104.9021:33.5747;-104.6784:...,10000.0,19000.0,NaN,NaN,ICE,MOD,AIRMET
4,WAUS44 KKCI 020245 DFWZ WA 020245 AIRMET ZUL...,2024-01-02T02:45:00Z,2024-01-02T08:45:00Z,NaN,NaN,NaN,NaN,NaN,ICE,1,AIRMET


In [1106]:
# replace with <br> in raw_text
airsigmet_data_df["raw_text"] = airsigmet_data_df["raw_text"].replace(r'\x07','<br>', regex=True)   # Used to replace \n that does not get decompressed as utf-8 and was converted as \x07

# To convert the points delimiting the areas into something Leaflet-friendly
for j in range(len(airsigmet_data_df)):
    test=airsigmet_data_df.iloc[j]['lon:lat points']
    if  pd.isna(test)!=True:    # Test if the cell is not NaN
        test1=test.split(';')
        for i in range(len(test1)):
            test1[i]=test1[i].split(':')    # Creates list of coordinates
            test1[i][0],test1[i][1]=float(test1[i][1]),float(test1[i][0])   # Swap lon:lat to lat:lon
        airsigmet_data_df.at[j,'lon:lat points']=test1

    # shift cells for missing column
    test2=airsigmet_data_df.iloc[j]['airsigmet_type']
    if  pd.isna(test2)==True:    # Test if the cell is NaN
        airsigmet_data_df.at[j,'airsigmet_type']=airsigmet_data_df.at[j,'severity']
        airsigmet_data_df.at[j,'severity']=airsigmet_data_df.at[j,'hazard']
        airsigmet_data_df.at[j,'hazard']=airsigmet_data_df.at[j,'movement_speed_kt']
        airsigmet_data_df.at[j,'movement_speed_kt']=airsigmet_data_df.at[j,'movement_dir_degrees']
        airsigmet_data_df.at[j,'movement_dir_degrees']=airsigmet_data_df.at[j,'max_ft_msl']
        airsigmet_data_df.at[j,'max_ft_msl']=airsigmet_data_df.at[j,'min_ft_msl']
        airsigmet_data_df.at[j,'min_ft_msl']="NaN"

  


airsigmet_data_df.rename(columns={"lon:lat points":"lat_lon_points"}, inplace=True) # Update the column name
airsigmet_data_df.head()

,raw_text,valid_time_from,valid_time_to,lat_lon_points,min_ft_msl,max_ft_msl,movement_dir_degrees,movement_speed_kt,hazard,severity,airsigmet_type
0,WSUS32 KKCI 020655 SIGC CONVECTIVE SIGMET.....,2024-01-02T06:55:00Z,2024-01-02T08:55:00Z,"[[49.0, -107.0], [31.79, -107.0], [31.78, -106...",NaN,NaN,NaN,NaN,CONVECTIVE,1,SIGMET
1,WSUS31 KKCI 020655 SIGE CONVECTIVE SIGMET 6...,2024-01-02T06:55:00Z,2024-01-02T08:55:00Z,"[[37.2917, -72.4317], [36.2246, -73.7076], [36...",NaN,21000.0,NaN,NaN,CONVECTIVE,LT-MOD,SIGMET
2,WSUS33 KKCI 020655 SIGW CONVECTIVE SIGMET 2...,2024-01-02T06:55:00Z,2024-01-02T08:55:00Z,"[[32.0135, -109.9843], [31.2, -108.1224], [31....",NaN,28000.0,NaN,NaN,CONVECTIVE,LT-MOD,SIGMET
3,WAUS45 KKCI 020245 SLCZ WA 020245 AIRMET ZUL...,2024-01-02T02:45:00Z,2024-01-02T09:00:00Z,"[[35.0385, -106.2056], [33.5747, -104.9021], [...",10000.0,19000.0,NaN,NaN,ICE,MOD,AIRMET
4,WAUS44 KKCI 020245 DFWZ WA 020245 AIRMET ZUL...,2024-01-02T02:45:00Z,2024-01-02T08:45:00Z,NaN,NaN,NaN,NaN,NaN,ICE,1,AIRMET


In [1107]:
# Convert the dataframe to a JSON format
# airsigmet_json = json.loads(json.dumps(list(airsigmet_data_df.T.to_dict().values())))   # https://stackoverflow.com/questions/39257147/convert-pandas-dataframe-to-json-format answer #10 Amir.S
# pprint(airsigmet_json, sort_dicts=False)

In [1108]:
# convert the dataframe to a dictionary then to a JSON string
airsigmet_string=json.dumps(list(airsigmet_data_df.T.to_dict().values()))

# open the file in write mode
# output_path = os.path.join("Resources", "airsigmet_data.json")
output_path = os.path.join("static", "airsigmet_data.json")
with open(output_path, "w") as file:
    # write the JSON string to the file
    file.write(airsigmet_string.replace("NaN","null"))

# file is automatically closed after the with block

__To download headwing and crosswind informations for all runways__ (Not used anymore: to be deleted)

In [1109]:
# # download the output of a query across multiple tables with the wind information relative to all runways.
# # Will be used to update the makers for all airports.
# # There are multiple rows per airport (one per runway).
# # More parameters can be added to complete the info on the popup window.

# load_dotenv()
# db_url = os.environ.get("link_render")


# query="""
# SELECT arpt_id, icao_id, arpt_name, apt_rwy.rwy_id, apt_rwy.rwy_len, apt_rwy_end.rwy_end_id, apt_rwy_end.true_alignment, metar.wind_dir_degrees, metar.wind_speed_kt, metar.wind_gust_kt,
# ROUND(metar.wind_speed_kt*sin(radians(apt_rwy_end.true_alignment - (metar.wind_dir_degrees :: INTEGER)))) AS "cross_wind",
# ROUND(metar.wind_speed_kt*cos(radians(apt_rwy_end.true_alignment - (metar.wind_dir_degrees :: INTEGER)))) AS "head_wind",
# ROUND(metar.wind_gust_kt*sin(radians(apt_rwy_end.true_alignment - (metar.wind_dir_degrees :: INTEGER)))) AS "gust_cross_wind",
# ROUND(metar.wind_gust_kt*cos(radians(apt_rwy_end.true_alignment - (metar.wind_dir_degrees :: INTEGER)))) AS "gust_head_wind"	
# FROM apt_rwy
# JOIN apt_base ON apt_base.site_no = apt_rwy.site_no
# JOIN apt_rwy_end ON apt_base.site_no = apt_rwy_end.site_no AND apt_rwy.rwy_id = apt_rwy_end.rwy_id
# FULL JOIN metar ON (RIGHT(metar.station_id, LENGTH(metar.station_id) - 1)) = apt_base.arpt_id
# WHERE facility_use_code='PU' AND arpt_status='O' AND site_type_code='A' AND (@(lat_decimal - metar.latitude) <1) AND (@(long_decimal - metar.longitude) <1) AND metar.wind_dir_degrees != 'VRB'
# """

# engine=create_engine(db_url)
# with engine.begin() as conn:
#     results=conn.execute(
#         text(query)
#     )

# rwy_wind_query_df = pd.DataFrame(results)


# # convert the dataframe to a dictionary then to a JSON string
# rwy_wind_string=json.dumps(list(rwy_wind_query_df.T.to_dict().values()))

# # open the file in write mode
# output_path = os.path.join("", "rwy_wind_data.json")
# with open(output_path, "w") as file:
#     # write the JSON string to the file
#     file.write(rwy_wind_string.replace("NaN","null"))

# # file is automatically closed after the with block

In [1110]:
# rwy_wind_query_df.head()

In [1111]:
# len(rwy_wind_query_df)

In [1112]:
# rwy_wind_query_df

__Retrieve all data to position the circles for all airports and create the popup text__

In [1113]:
# download the output of a consolidated query across multiple tables with all information pertinent to an airport.
# Will be used to update the makers for all airports.
# There are multiple rows per airport (one per runway).
# More parameters can be added to complete the info on the popup window.
# Will be used to consolidate the information in the popups

load_dotenv()
db_url = os.environ.get("link_render")

query="""
-- Query to get METAR information for each airport as a VIEW
DROP VIEW IF EXISTS all_circles_view;
CREATE VIEW all_circles_view AS
SELECT 
arpt_id, icao_id, metar.station_id, arpt_name, apt_rwy.rwy_id, apt_rwy.rwy_len, lat_decimal, metar.latitude, long_decimal, metar.longitude, metar.observation_time, metar.flight_category, metar.raw_text, elev, metar.visibility_statute_mi, metar.cloud_base_ft_agl
-- , metar.wind_speed_kt
FROM apt_rwy
JOIN apt_base ON apt_base.site_no = apt_rwy.site_no
FULL JOIN metar ON (RIGHT(metar.station_id, LENGTH(metar.station_id) - 1)) = apt_base.arpt_id
WHERE facility_use_code='PU' AND site_type_code='A' AND arpt_status='O' AND
CASE
	WHEN metar.station_id IS NOT NULL
	THEN (@(lat_decimal - metar.latitude) <1) AND (@(long_decimal - metar.longitude) <1) AND site_type_code='A'
	
	WHEN metar.station_id IS NULL
	THEN site_type_code='A'
END;


-- Query to get wind information for each runway as a VIEW
DROP VIEW IF EXISTS all_rwy_xwind_view;
CREATE VIEW all_rwy_xwind_view AS
SELECT arpt_id, icao_id, arpt_name, apt_base.tpa, apt_rwy.rwy_id, apt_rwy.rwy_len, apt_rwy.rwy_width, apt_rwy_end.rwy_end_id, apt_rwy_end.true_alignment, apt_rwy_end.right_hand_traffic_pat_flag, metar.wind_dir_degrees, metar.wind_speed_kt, metar.wind_gust_kt,
	ROUND(metar.wind_speed_kt*sin(radians(apt_rwy_end.true_alignment - (metar.wind_dir_degrees :: INTEGER)))) AS "cross_wind",
	ROUND(metar.wind_speed_kt*cos(radians(apt_rwy_end.true_alignment - (metar.wind_dir_degrees :: INTEGER)))) AS "head_wind",
	ROUND(metar.wind_gust_kt*sin(radians(apt_rwy_end.true_alignment - (metar.wind_dir_degrees :: INTEGER)))) AS "gust_cross_wind",
	ROUND(metar.wind_gust_kt*cos(radians(apt_rwy_end.true_alignment - (metar.wind_dir_degrees :: INTEGER)))) AS "gust_head_wind"	
FROM apt_rwy_end
FULL JOIN apt_base ON apt_base.site_no = apt_rwy_end.site_no
FULL JOIN apt_rwy ON apt_base.site_no = apt_rwy.site_no AND apt_rwy.rwy_id = apt_rwy_end.rwy_id
FULL JOIN metar ON (RIGHT(metar.station_id, LENGTH(metar.station_id) - 1)) = apt_base.arpt_id
WHERE facility_use_code='PU' AND arpt_status='O' AND
CASE -- by not have the CASE, we were not providing the rwy length of the airports without METAR
	WHEN metar.station_id IS NOT NULL
	AND metar.wind_dir_degrees != 'VRB'
	THEN (@(lat_decimal - metar.latitude) <1) AND (@(long_decimal - metar.longitude) <1) AND site_type_code='A'
	
	WHEN metar.station_id IS NULL
	THEN site_type_code='A'
END;


-- Merge the two VIEWS
SELECT 	all_circles_view.arpt_id, all_circles_view.icao_id, all_circles_view.station_id, all_circles_view.arpt_name, all_circles_view.lat_decimal, all_circles_view.long_decimal, all_circles_view.elev, all_rwy_xwind_view.tpa,
		all_circles_view.rwy_id, all_rwy_xwind_view.rwy_end_id, all_rwy_xwind_view.rwy_len, all_rwy_xwind_view.rwy_width, all_rwy_xwind_view.right_hand_traffic_pat_flag,
		all_rwy_xwind_view.cross_wind, all_rwy_xwind_view.head_wind, all_rwy_xwind_view.gust_cross_wind, all_rwy_xwind_view.gust_head_wind,		
		all_rwy_xwind_view.true_alignment,
		all_rwy_xwind_view.wind_dir_degrees, all_rwy_xwind_view.wind_speed_kt, all_rwy_xwind_view.wind_gust_kt,
		all_circles_view.flight_category,  all_circles_view.visibility_statute_mi, all_circles_view.cloud_base_ft_agl,		
		all_circles_view.observation_time, all_circles_view.raw_text
FROM all_circles_view
FULL JOIN all_rwy_xwind_view ON all_circles_view.arpt_id = all_rwy_xwind_view.arpt_id
AND all_circles_view.rwy_id=all_rwy_xwind_view.rwy_id
WHERE all_circles_view.rwy_id NOT LIKE 'H%'
"""


engine=create_engine(db_url)
with engine.begin() as conn:
    results=conn.execute(
        text(query)
    )

airport_info_query_raw_df = pd.DataFrame(results)


# # convert the dataframe to a dictionary then to a JSON string		# The section below will be moved to be used on the consolidated DF.
# airport_info_string=json.dumps(list(airport_info_query_raw_df.T.to_dict().values()))

# # open the file in write mode
# output_path = os.path.join("", "airport_info_data.json")
# with open(output_path, "w") as file:
#     # write the JSON string to the file
#     file.write(rwy_wind_string.replace("NaN","null"))

# # file is automatically closed after the with block

In [1114]:
# airport_info_query_raw_df.head()

In [1115]:
airport_info_query_raw_df.columns

Index(['arpt_id', 'icao_id', 'station_id', 'arpt_name', 'lat_decimal',
       'long_decimal', 'elev', 'tpa', 'rwy_id', 'rwy_end_id', 'rwy_len',
       'rwy_width', 'right_hand_traffic_pat_flag', 'cross_wind', 'head_wind',
       'gust_cross_wind', 'gust_head_wind', 'true_alignment',
       'wind_dir_degrees', 'wind_speed_kt', 'wind_gust_kt', 'flight_category',
       'visibility_statute_mi', 'cloud_base_ft_agl', 'observation_time',
       'raw_text'],
      dtype='object')

In [1116]:
# Consolidation of the database with only one row per airport regardless of the number of runways

airport_info_query_df=pd.DataFrame(columns=['arpt_id', 'icao_id', 'station_id', 'arpt_name', 'lat_decimal',
       'long_decimal', 'elev', 'tpa', 'rwy_id', 'rwy_end_id', 'rwy_len',
       'rwy_width', 'right_hand_traffic_pat_flag', 'cross_wind', 'head_wind',
       'gust_cross_wind', 'gust_head_wind', 'true_alignment',
       'wind_dir_degrees', 'wind_speed_kt', 'wind_gust_kt', 'flight_category',
       'visibility_statute_mi', 'cloud_base_ft_agl', 'observation_time',
       'raw_text'])

airport_info_query_df["popup_text"]=np.nan

first_row=True
first_rwy=True

a=len(airport_info_query_raw_df)
print(f"View length: {a}")
for i in range (a):
    if first_row:
       k=i
       airport_info_query_df.loc[airport_info_query_raw_df.index[i]] = airport_info_query_raw_df.iloc[i]
       popup_text="""<div class="arpt_id"> """+airport_info_query_raw_df.iloc[i]["arpt_id"]+'</div>'

       if airport_info_query_raw_df.iloc[i]["icao_id"] is not None:
           popup_text=popup_text+"""<div class="icao_id"> / """+airport_info_query_raw_df.iloc[i]["icao_id"]+'</div>'

       popup_text=popup_text+'<br>'+"""<div class="arpt_name"> """+airport_info_query_raw_df.iloc[i]["arpt_name"]+'</div>'
       popup_text=popup_text+'<br>'+"""<div class="elev">Airport Elevation: """+str(int(airport_info_query_raw_df.iloc[i]["elev"]))+""" (ft MSL)</div>"""

       if pd.isna(airport_info_query_raw_df.iloc[i]["tpa"])==False:
           popup_text=popup_text+'<br>'+"""<div class="tpa">TPA: """+str(int(airport_info_query_raw_df.iloc[i]["elev"]+int(airport_info_query_raw_df.iloc[i]["tpa"])))+""" (ft MSL)</div>"""
       else:
           popup_text=popup_text+'<br>'+"""<div class="tpa">TPA (estimated): """+str(int(airport_info_query_raw_df.iloc[i]["elev"]+1000))+""" (ft MSL)</div>"""
             

       if pd.isna(airport_info_query_raw_df.iloc[i]["visibility_statute_mi"])==False:
           popup_text=popup_text+'<br>'+"""<div class="visibility_statute_mi">Airport visibility: """+airport_info_query_raw_df.iloc[i]["visibility_statute_mi"]+' (SM)</div>'

       if pd.isna(airport_info_query_raw_df.iloc[i]["cloud_base_ft_agl"])==False:
           popup_text=popup_text+'<br>'+"""<div class="cloud_base_ft_agl">Airport ceiling: """+str(airport_info_query_raw_df.iloc[i]["cloud_base_ft_agl"])+' (ft AGL)</div>'

       first_row=False

    if first_rwy:
        popup_text=popup_text+'<br>'+"""<div class="rwy_id"> Runway """+airport_info_query_raw_df.iloc[i]["rwy_id"]+' </div>'
        if pd.isna(airport_info_query_raw_df.iloc[i]["rwy_len"])==False:
            popup_text=popup_text+'<br>'+"""<div class="rwy_len"> Runway length: """+str(int(airport_info_query_raw_df.iloc[i]["rwy_len"]))+' (ft)</div>'
        if pd.isna(airport_info_query_raw_df.iloc[i]["rwy_width"])==False:
            popup_text=popup_text+'<br>'+"""<div class="rwy_width"> Runway width: """+str(int(airport_info_query_raw_df.iloc[i]["rwy_width"]))+' (ft)</div>'
        first_rwy=False

    if (pd.isna(airport_info_query_raw_df.iloc[i]["raw_text"])==False & pd.isna(airport_info_query_raw_df.iloc[i]["true_alignment"])==False) | (airport_info_query_raw_df.iloc[i]["right_hand_traffic_pat_flag"]=='Y'):
        if pd.isna(airport_info_query_raw_df.iloc[i]["rwy_end_id"])==False:
            popup_text=popup_text+'<br>'+"""<div class="rwy_end_id"> """+airport_info_query_raw_df.iloc[i]["rwy_end_id"]+' :</div>'
            if airport_info_query_raw_df.iloc[i]["right_hand_traffic_pat_flag"]=='Y':
                popup_text=popup_text+'<br>'+"""<div class="RP"> RP </div>"""


        if pd.isna(airport_info_query_raw_df.iloc[i]["cross_wind"])==False:
            popup_text=popup_text+'<br>'+"""<div class="cross_wind">Crosswind (neg. is from right): """+str(int(airport_info_query_raw_df.iloc[i]["cross_wind"]))+' (kt)</div>'

        if pd.isna(airport_info_query_raw_df.iloc[i]["head_wind"])==False:
            popup_text=popup_text+'<br>'+"""<div class="head_wind">Headwind (neg. is headwind): """+str(int(airport_info_query_raw_df.iloc[i]["head_wind"]))+' (kt)</div>'

        if pd.isna(airport_info_query_raw_df.iloc[i]["gust_cross_wind"])==False:
            popup_text=popup_text+'<br>'+"""<div class="gust_cross_wind">Gust crosswind (neg. is from right): """+str(int(airport_info_query_raw_df.iloc[i]["gust_cross_wind"]))+' (kt)</div>'

        if pd.isna(airport_info_query_raw_df.iloc[i]["gust_head_wind"])==False:
            popup_text=popup_text+'<br>'+"""<div class="gust_head_wind">Gust headwind (neg. is headwind): """+str(int(airport_info_query_raw_df.iloc[i]["gust_head_wind"]))+' (kt)</div>'


    if i<a-1:
        if airport_info_query_raw_df.iloc[i+1]["arpt_id"]!=airport_info_query_raw_df.iloc[i]["arpt_id"]:
            if pd.isna(airport_info_query_raw_df.iloc[i]["raw_text"])==False:
                popup_text=popup_text+'<br>'+"""<div class="raw_text"> """+airport_info_query_raw_df.iloc[i]["raw_text"]+'</div>'
                popup_text=popup_text+'<br>'+"""<div class="observation_time"> """+airport_info_query_raw_df.iloc[i]["observation_time"]+'</div>'
            else:
                popup_text=popup_text+'<br> <div class="raw_text">No weather information</div>'

            airport_info_query_df.at[k,"popup_text"]=popup_text
            # print(f"k={k}, i={i}")
            # print(popup_text)
            first_row=True
            first_rwy=True
        elif airport_info_query_raw_df.iloc[i+1]["rwy_id"]!=airport_info_query_raw_df.iloc[i]["rwy_id"]:
            first_rwy=True

    if i==a-1:
        airport_info_query_df.at[k,"popup_text"]=popup_text




# convert the dataframe to a dictionary then to a JSON string		# The section below will be moved to be used on the consolidated DF.
airport_info_string=json.dumps(list(airport_info_query_df.T.to_dict().values()))

# open the file in write mode
output_path = os.path.join("static", "airport_info_full_data.json")
with open(output_path, "w") as file:
    # write the JSON string to the file
    file.write(airport_info_string.replace("NaN","null"))

# file is automatically closed after the with block
        


# airport_info_query_df




View length: 13570


In [1117]:
# airport_info_query_df[airport_info_query_df['arpt_id']=='BLU']

__Write the json result files to AWS S3__

In [ ]:



load_dotenv()
# key1 = os.environ.get("AWS_ACCESS_KEY_ID")
# key2 = os.environ.get("AWS_SECRET_ACCESS_KEY")

def upload_file(file_name, bucket, object_name=None):   # Inspired from https://docs.aws.amazon.com/AmazonS3/latest/userguide/example-walkthroughs-managing-access-example1.html
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client(
        's3',
        # aws_access_key_id= os.environ.get("AWS_ACCESS_KEY_ID"),
        # aws_secret_access_key=os.environ.get("AWS_SECRET_ACCESS_KEY") )
        aws_access_key_id= os.environ.get("AWS_ACCESS_KEY_ID"),
        aws_secret_access_key= os.environ.get("AWS_SECRET_ACCESS_KEY"))

    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True


upload_file("static/airsigmet_data.json",'ga-weather', "airsigmet_data.json")
upload_file("static/airport_info_full_data.json",'ga-weather', "airport_info_full_data.json")
